# 1. polglot-ko-1.3b를 squarelike/sharegpt_deepl_ko_translation로 파인 튜닝한 모델

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "EleutherAI/polyglot-ko-5.8b"
model_dir = "./results/poly-ko-5.8b-translate"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
model = AutoModelForCausalLM.from_pretrained(model_dir, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [3]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [4]:
def gen(lan="en", x=""):
    if (lan == "ko"):
        prompt = f"### 한국어: {x}</끝>\n### 영어:"
    else:
        prompt = f"### 영어: {x}</끝>\n### 한국어:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=2048,
        temperature=0.001,
        no_repeat_ngram_size=10,
        early_stopping=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    return tokenizer.decode(gened[0]).replace(prompt+" ", "")

In [5]:
gen(lan="en", x="NATO summit keeps focus on Ukraine as Russia's war continues. Here's what you need to know")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'NATO 회담은 우크라이나의 전쟁으로 계속 집중되고 있습니다. 알아야 할 사항입니다.</끝>'

In [6]:
gen(lan="en", x="Russian defense ministry says Wagner has handed over tanks and other weapons")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'러시아 국방부는 와그너가 탱크와 기타 무기를 인계했다고 말합니다.</끝>'

In [7]:
gen(lan="en", x="You’re misunderstanding right now. Hear me out.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'지금까지 오해하고 있습니다. 제가 말해 주세요.</끝>'

In [8]:
gen(lan="en", x="New York. CNN. The Federal Aviation Administration has certified for testing a vehicle that a California startup describes as a flying car — the first fully electric vehicle that can both fly and travel on roads to receive US government approval. Alef Automotive said that its vehicle/aircraft, dubbed the “Model A,” is the first flying vehicle that is drivable on public roads and able to park like a normal car. It also has vertical takeoff and landing capabilities. It apparently will be able to carry one or two occupants and will have a road-range of 200 miles and a flying range of 110 miles. The company expects to sell the vehicle for $300,000 each with the first delivery by projected for the end of 2025. The FAA confirmed that it has issued the company a special airworthiness certificate, allowing for limited purposes that include exhibition, research and development. Numerous companies are working on all-electric VTOLs, which stands for vehicle takeoff and landing aircraft. The FAA said that Alef is “not the first aircraft of its kind” to get a special airworthiness certificate. However, Alef noted that its vehicle is different because of its ability to function both on roads and in the air, to appear like a normal car and to park in a normal parking space.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'뉴욕. 연방항공당국은 캘리포니아 정부 기관이 미국 정부에 비행할 수 있는 차량을 비행할 수 있는 차량을 비행할 때 도로를 비행할 수 있는 차량입니다. Alef Automotive Af you wouldn the first 자동차는 일반 도로 운전 가능한 최초 운전 가능한 비행 가능한 비행 차량이라고 말했습니다. 또한 수직 착륙과 착륙 기능도 있습니다. 이는 승객 수를 운반할 수 있으며, 도로 주행 가능한 200마일 범위 및 비행 범위를 보유하고 있습니다. 이 회사는 2025년에 예상되는 차량을 판매할 예정이며, 차량을 판매할 예정이며, 차량의 화물을 판매할 수 있습니다. FAA는 회사가 이 항공 인증을 허가받았으며 전시 및 개발, 연구 및 개발 및 개발을 포함한 제한이 있습니다. 많은 회사는 전에 항공 착륙 및 착륙을 위해 노력하는 모든 전기 VTOL을 의미합니다. FAA는 Alef는 "알레의 첫 항공기의 첫 비행기를 받을 수 있는 최초의 항공의 첫 번째 항공기를 사용했습니다. 그러나 Alef는 도로에서 두 차량에서 운전할 수 있는 기능으로 인해 차량과 같은 차량이라는 차량으로 인해 차량을 주차할 수 있는 기능이라고 말했습니다.</끝>'